# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Didier Camilo Olarte Gonzalez <br>
Url: https://github.com/Did13r/VIU-Algoritmos-de-optimizacion.git <br>
Problema:
> 1. Sesiones de doblaje <br>



Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. 

Número de actores: 10  <br>
Número de tomas : 30

(*) La respuesta es obligatoria







(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

En terminos de la programacion de las tomas a ser grabadas, las posibilidades de organizacion son las mismas en ambos casos y estarian dadas por 30!, de manera que elaboramos nuestra lista de tomas, seleccionando de una en una, sin repetir, hasta agotar las 30.
Incluso al introducir la restriccion de maximo 6 tomas diarias, el orden general de grabacion permaneceria inalterado, unicamente seria segmentado en grupos de a seis. Lo que si se ve impactado en definitiva es el costo, pues, sin restricciones, podemos grabar todo en un solo dia y pagar unicamente a los 10 actores por esa jornada, resultando en un coste de 10(1 Cheque por actor), en cambio, esta restriccion nos obliga a llamar a ciertos actores en repetidas ocaciones, elevando el coste.
En ese sentido pasamos de tener 1 solo dia de grabacion a tener 5.


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Puedo hacer uso de listas o de dataframes, en los que voy guardando de forma iterativa las tomas seleccionas por sesion de grabacion, y otra en la que lleve registro de los actores involucrados en ella.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Una posible formulación en notación matemática de la función objetivo para minimizar el costo total de los servicios de los actores de doblaje podría ser la siguiente:

Minimizar i=1∑n ​Ci​xi​

donde n es el número total de días en que se graban tomas, Ci​ es el costo por día para todos los actores que deben desplazarse al estudio de grabación en el día i, y xi​ es una variable binaria que toma el valor 1 si hay al menos una toma programada para el día i y 0 en caso contrario.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [ ]:
import pandas as pd
import numpy as np
import random
from itertools import permutations

#Leer el fichero datos_A.csv - yo di un formato ligeramente diferente el dataset provisto
file_path = 'datos_A.csv'
df = pd.read_csv(file_path)

#Mostar el DataFrame
#print(df)


#funcion que genera lista de todas las combinaciones posibles de las 30 (o n) tomas 
#cada combinacion se corresponde con una programacion general de grabacion 
#es decir (el orden de grabacion de todas las tomas sin agrupar por dias)
def generar_combinaciones(n):
    tomas = list(range(1, n+1))
    combinaciones = [list(comb) for comb in permutations(tomas)]
    return combinaciones

# n deberia ser 30 que es el numero de tomas pero despues de 9 mi laptop no da mas
# asi que pruebo el algoritmo con n<9
n = 7 
combinaciones = generar_combinaciones(n)
#print(combinaciones)


#funcion que evalua cada combinacion y calcula el costo de la solucion dada
def Generate_BF_solution(df, combination):
    
    #dataframe que relaciona la tomas de la combinacion evaluado con los actores que participan en ella
    total_actors_df = pd.DataFrame({'Toma': combination, 'Total Actors': [df.loc[df['Toma'] == toma, 'ActoresPorToma'].iloc[0] for toma in combination]})

    #conjuntos que se emplean para llevar cuenta de la participacion diaria y acumulada
    actor_participation = set()
    full_actor_participation = set()

    #listas que se emplean para llevar cuenta de la programacion de tomas diaria y acumulada
    recording_schedule_takes = []
    Recordig_Schedule_Full = []
    
    #varaibles de control de flujo del condicional
    recording_days = 0
    row_count = len(total_actors_df)
    
    #costo de la solucion evaluada
    cost = 0

    take_C = total_actors_df.iloc[0]['Toma']

    while row_count > 0:

        #Seleccion de la primera toma del dataframe, se guarda en toma_A, 
        #cuenta los actores en ella, para luego eliminarla del dataframe de referencia
        #y le añade a la programacion de grabacion
        
        take_A = take_C

        index_to_drop = total_actors_df[total_actors_df['Toma'] == take_A].index

        total_actors_df = total_actors_df.drop(index_to_drop)

        actors = df[df['Toma'] == take_A].iloc[0, 1:]

        for actor, present in actors.items():
            if present == 1:
                actor_participation.add(actor)

        recording_schedule_takes.append(take_A)
        
        #se evalua el dataframe de referenca y si solo queda una toma por grabar
        #cuenta los actores participantes, los agrega a la lista, y hace la cuenta
        #de los totales
        if len(total_actors_df) == 1:

            recording_days += 1
        
            last_take = total_actors_df.iloc[0].loc['Toma']
            recording_schedule_takes.append(last_take)

            actors = df[df['Toma'] == last_take].iloc[0, 1:]

            for actor, present in actors.items():
                if present == 1:
                    actor_participation.add(actor)

            #print(f"Actor Participation per DAY: {actor_participation}")

            full_actor_participation.update(actor_participation)
            actor_count = len(actor_participation)
            actor_participation = set()

            #print(f"Recording Schedule until DAY: {recording_schedule_takes}")
            
            Recordig_Schedule_Full.append(recording_schedule_takes)
            recording_schedule_takes = []
            #print(f"Recording Schedule Acumulated: {Recordig_Schedule_Full}")

            actors_recording_day = pd.DataFrame({'Day': [recording_days], 'Actor Count': [actor_count]})

            cost += actor_count

            #print(actors_recording_day)
            #print("----- end --------------------------------of-------------------------------- day ---------")
            #print("------------------------------------------------------------------------------------------\n\n")
            break

        #asigna a B la nueva primera toma del dataframe con las tomas restantes y a C la que le sigue
        take_B = total_actors_df.iloc[0]['Toma']
        take_C = total_actors_df.iloc[1]['Toma']


        #cuenta los actores en ella, para luego eliminarla del dataframe de referencia
        #y le añade a la programacion de grabacion
        actors = df[df['Toma'] == take_B].iloc[0, 1:]

        for actor, present in actors.items():
            if present == 1:
                actor_participation.add(actor)

        recording_schedule_takes.append(take_B)

        index_to_drop = total_actors_df[total_actors_df['Toma'] == take_B].index

        total_actors_df = total_actors_df.drop(index_to_drop)

        #se actualiza la varible de control del while
        row_count = len(total_actors_df)

        #se evalua si dataframe de referenca y si van 6 tomas grabadas
        #cuenta los actores participantes, los agrega a la lsita, y hace la cuenta
        #de los totales diarios
        if len(recording_schedule_takes) % 6 == 0 :

            recording_days += 1

            #print(f"Actor Participation per DAY: {actor_participation}")

            full_actor_participation.update(actor_participation)
            actor_count = len(actor_participation)
            actor_participation = set()

            #print(f"Recordig Schedule until DAY: {recording_schedule_takes}")
            
            Recordig_Schedule_Full.append(recording_schedule_takes)
            recording_schedule_takes = []
            
            #print(f"Recordig Schedule Acumulated: {Recordig_Schedule_Full}")

            actors_recording_day = pd.DataFrame({'Day': [recording_days], 'Actor Count': [actor_count]})

            cost += actor_count

            #print(actors_recording_day)
            #print("----end ------------------------------of---------------------------------- day ---------")

    return full_actor_participation, Recordig_Schedule_Full, actors_recording_day, cost


#implementacion del algoritmo para generar una lista con las soluciones y escoge la mejor 'menor costo'

#Lsita que guarda todas las soluciones generadas
solutions = []

#genera las soluciones llamando iterativamente la funcion tantas veces como haya combinaciones
for combination in combinaciones:
    full_actor_participation, Recordig_Schedule_Full, actors_recording_total, cost = Generate_BF_solution(df, combination)
    
    #Guarda la solucion particular y la agrega al listado
    solution = {
        'full_actor_participation': full_actor_participation,
        'Recordig_Schedule_Full': Recordig_Schedule_Full,
        'actors_recording_total': actors_recording_total,
        'cost': cost
    }
    solutions.append(solution)

#selecciona del listado aquella solucion con el menor costo
best_solution = min(solutions, key=lambda x: x['cost'])

print("Best solution:")
print(best_solution)


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

la complejidad del algoritmo es del orden factorial pues el enfoque de fuerza bruta nos obliga a recorrer todo el espacio de soluciones que estaria compuesto por todas las posibles combinaciones de orden de tomas. Habria que calcular el costo de rodaje diario y total (que es dependiente de la seleccion particular de tomas y actores implicados en ellas) para 30! ≈ 2.65 x 10^32 esa cantidad de tomas y luego compararlas para obtener la solucion que ofrezca el menos coste.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Planteo un algoritmo basado en ordenacon por tomas, seleccionandolas segun la cantidad de actores participantes.
La idea es generar la programacion de grabacion con las tomas con mayor numero de actores e ir descendiento progresivamente. Para ello se escoge al azar una toma entre aquellas con el reparto mas grande. <br>
Cada toma subsecuente es escogida con base al numero de actores que tenga en comun con la toma previa. Para esto se calcula una matriz de comunalidades de actores para todas la tomas y se emplea como criterio de filtro de las tomas suceptibles a ser selecciondas el IC (indice de comunalidad) entre 2 tomas dadas. En caso de existir multiples IC de igual magnitud, se hace una seleccion aleatoria. <br>
Este proceso se repite ciclicamente hasta completar 6 tomas por dia, momento en el cual se calucla la paritcipacion actoral total y con ello el costo por dia.<br>
Se itera el paso anterior hasta terminar todas las tomas, llevando cuenta del total de dias y el acumulado del costo. Es este ultima costo total el valor a minimizar.<br>
Finalmente, se llama esta funcion generadora de soluciones un numero n de veces (1000), para generar una poblacion de soluciones entre las cuales se escoge la mejor (el menor costo).<br>
<br>
Este es un algoritmo tipo GRASP pues busca tomar la desicion optima en cada paso (Greedy), pero comtempla elementos de aleatorizacion al escoger al azar entre opciones equivalentes lo que genera siempre soluciones diferentes, incluso desde la eleccion de la primera toma, lo cual hace que funcione como un metaheuristica iterativa de multiarranque

In [ ]:
import pandas as pd
import numpy as np
import random

file_path = 'datos_A.csv'
df = pd.read_csv(file_path)

#calcular la matrix de comunalidades entre todas las tomas
n_takes = len(df)
n_actors = len(df.columns) - 1
communality_matrix = np.zeros((n_takes, n_takes))

for i in range(n_takes):
    for j in range(n_takes):
      
        matching_actors = (df.iloc[i, 1:] & df.iloc[j, 1:] == 1).sum()
        
        union_actors = ((df.iloc[i, 1:] == 1) | (df.iloc[j, 1:] == 1)).sum()
       
        CI = matching_actors / union_actors
       
        communality_matrix[i,j] = CI
        communality_matrix[j,i] = CI
        

communality_df = pd.DataFrame(communality_matrix,
                              index=df['Toma'],
                              columns=df['Toma'])

#print('matrix de comunalidades:')
#print(communality_df)

#funcion que evalua cada combinacion y calcula el costo de la solucion dada
def Generate_solution(df, communality_df):
    
    #dataframe que relaciona la tomas de la combinacion evaluada con los actores que participan en ella
    #ordenandolo en orden decreciente segun numero de actores
    total_actors_df = pd.DataFrame({'Toma': df['Toma'], 'Total Actors': df['ActoresPorToma']})
    total_actors_df = total_actors_df.sort_values(by='Total Actors', ascending=False)

    Actor_per_take_Ordered_df = total_actors_df[['Toma', 'Total Actors']].reset_index(drop=True)

    #conjuntos que se emplean para llevar cuenta de la participacion diaria y acumulada
    actor_participation = set()
    full_actor_participation = set()

    #listas que se emplean para llevar cuenta de la programacion de tomas diaria y acumulada
    recording_schedule_takes = []
    Recordig_Schedule_Full = []
    
    #varaibles de control de flujo del condicional
    recording_days = 0
    row_count = len(Actor_per_take_Ordered_df)
    
    #costo de la solucion evaluada
    cost = 0
    
    #creacion de dataframe con tomas con el mayor numero de actores del cual se
    #selecciona una al azar y se asigna a toma_c para inicializarla
        
    highest_actor_count = Actor_per_take_Ordered_df["Total Actors"].max()

    highest_actor_count_df = Actor_per_take_Ordered_df[Actor_per_take_Ordered_df["Total Actors"] == highest_actor_count]

    take_C = random.choice(highest_actor_count_df["Toma"].values)

    while row_count > 0:
        
        #Seleccion de la primera toma del dataframe, se guarda en toma_A, 
        #cuenta los actores en ella, para luego eliminarla del dataframe de referencia
        #y le añade a la programacion de grabacion

        take_A = take_C         
        
        index_to_drop = Actor_per_take_Ordered_df[Actor_per_take_Ordered_df['Toma'] == take_A].index

        Actor_per_take_Ordered_df = Actor_per_take_Ordered_df.drop(index_to_drop)
        
        recording_schedule_takes.append(take_A)
        
        
        #cuenta los actores en ella, para luego eliminarla del dataframe de referencia
        #y le añade a la programacion de grabacion
        actors = df[df['Toma'] == take_A].iloc[0, 1:]

        for actor, present in actors.items():
            if present == 1:
                actor_participation.add(actor)
                
        #se evalua el dataframe de referenca y si solo queda una toma por grabar
        #cuenta los actores participantes, los agrega a la lista, y hace la cuenta
        #de los totales
        if len(Actor_per_take_Ordered_df) == 1:
            
            recording_days += 1
            
            last_take = Actor_per_take_Ordered_df.iloc[0].loc['Toma']
            recording_schedule_takes.append(last_take)
            
            actors = df[df['Toma'] == last_take].iloc[0, 1:]

            for actor, present in actors.items():
                if present == 1:
                    actor_participation.add(actor)
                    
            #print(f"Actor Participation per DAY: {actor_participation}")
            
            full_actor_participation.update(actor_participation)
            actor_count = len(actor_participation)
            actor_participation = set()
            
            #print(f"Recordig Schedule until DAY: {recording_schedule_takes}")
            Recordig_Schedule_Full.append(recording_schedule_takes)
            recording_schedule_takes = []
            #print(f"Recordig Schedule Acumulated: {Recordig_Schedule_Full}")
            
            actors_recording_day = pd.DataFrame({'Day': [recording_days], 'Actor Count': [actor_count]})
            
            cost += actor_count
            
            #print(actors_recording_day)
            #print("----- end --------------------------------of-------------------------------- day ---------")
            #print("------------------------------------------------------------------------------------------\n\n")
            break

        #escoger al azar una toma con la comunalidad mas alta dentro de las tomas restantes 
        #y asiganrla a toma_B y removerla de a lista de tomas
        row = communality_df.loc[take_A].drop(take_A)
        valid_takes = Actor_per_take_Ordered_df['Toma']
        row = row[row.index.isin(valid_takes)]
              
        max_value = row.max()
  
        max_value_indices = row[row == max_value].index

        take_B = random.choice(max_value_indices)

        max_value_indices_list = max_value_indices.tolist()

        max_value_indices_list.remove(take_B)

        #si no hay mas tomas entre las de comunalidad mas alta
        #escoger aleatoriamente etre las que les siguen y
        #asignarla a toma_C para luego removerla de la lista de disponibles
        if len(max_value_indices_list) > 0:
           
            take_C = random.choice(max_value_indices_list)
        else:
          
            row = communality_df.loc[take_A].drop(take_A)
            valid_takes = Actor_per_take_Ordered_df['Toma']
            row = row[row.index.isin(valid_takes)]
          
            unique_values = row.unique()
            unique_values.sort()
            unique_values = unique_values[::-1]

            second_highest_value = unique_values[1]

            scnd_value_indices = row[row == second_highest_value].index
           
            take_C = random.choice(scnd_value_indices)

        #cuenta los actores en la toma B, para luego eliminarla del dataframe de referencia
        #y le añade a la programacion de grabacion
        actors = df[df['Toma'] == take_B].iloc[0, 1:]

        for actor, present in actors.items():
            if present == 1:
                actor_participation.add(actor)

        recording_schedule_takes.append(take_B)
        
        index_to_drop = Actor_per_take_Ordered_df[Actor_per_take_Ordered_df['Toma'] == take_B].index

        Actor_per_take_Ordered_df = Actor_per_take_Ordered_df.drop(index_to_drop)
        
        #se actualiza la varible de control del while        
        row_count = len(Actor_per_take_Ordered_df)
        
        #se evalua si dataframe de referenca y si van 6 tomas grabadas
        #cuenta los actores participantes, los agrega a la lista, y hace la cuenta
        #de los totales diarios     
        if len(recording_schedule_takes) % 6 == 0 :
            
            recording_days += 1
            
            #print(f"Actor Participation per DAY: {actor_participation}")
            
            full_actor_participation.update(actor_participation)
            actor_count = len(actor_participation)
            actor_participation = set()
            
            #print(f"Recordig Schedule until DAY: {recording_schedule_takes}")
            Recordig_Schedule_Full.append(recording_schedule_takes)
            recording_schedule_takes = []
            #print(f"Recordig Schedule Acumulated: {Recordig_Schedule_Full}")

            actors_recording_day = pd.DataFrame({'Day': [recording_days], 'Actor Count': [actor_count]})
            
            cost += actor_count
            
            #print(actors_recording_day)
            #print("----end ------------------------------of---------------------------------- day ---------")
            
            
    return full_actor_participation, Recordig_Schedule_Full, actors_recording_total, cost
  
    
    
# implementacion del algoritmo para generar una poblacion de 1000 soluciones y escoger la mejor 'menor costo'


# cantidad de solutiones a generar
num_solutions = 1000

# lista para alamcenar las soluciones
solutions = []

# generar las soluciones
for i in range(num_solutions):
    
    full_actor_participation, Recordig_Schedule_Full, actors_recording_total, cost = Generate_solution(df, communality_df)
    
    
    solution = {
        'full_actor_participation': full_actor_participation,
        'Recordig_Schedule_Full': Recordig_Schedule_Full,
        'actors_recording_total': actors_recording_total,
        'cost': cost
    }
    solutions.append(solution)

# escoger la solucion con el menor costo
best_solution = min(solutions, key=lambda x: x['cost'])

#mostrar en pantalla
print("Best solution:")
print(best_solution)


(*)Calcula la complejidad del algoritmo

Respuesta

para encontrar una solucion es preciso calcular la matriz de cumunalidades el costo computacional asociado es de al menos n*n siendo n la cantidad de tomas (30).<br> 
Ya en el interior de la funcion tenemos el while que cicla n veces y en su interior un par de for loops que ciclan 10 veces(cantidad de actores), lo que nos da una complejidad de 10n. al interar el llamando a la funcion para generar una poblacion de soluciones de la cual escoger la mejor, se multiplica ese 10n por dicha cantidad, en este caso 1000.
Esto nos da 10000n. <br>  Si bien la complejidad de la matirz de comunalidaddes es cuadratica, el coeficiente que se obtiene al construir un espacio de soluciones diverso y numeroso, puede impactar de forma mas significativa el costo computacional de ejecutar el algoritmo.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [ ]:
data = {
    'Toma': list(range(1, 31)),
    'Actor1': np.random.randint(2, size=30),
    'Actor2': np.random.randint(2, size=30),
    'Actor3': np.random.randint(2, size=30),
    'Actor4': np.random.randint(2, size=30),
    'Actor5': np.random.randint(2, size=30),
    'Actor6': np.random.randint(2, size=30),
    'Actor7': np.random.randint(2, size=30),
    'Actor8': np.random.randint(2, size=30),
    'Actor9': np.random.randint(2, size=30),
    'Actor10': np.random.randint(2, size=30)
}

df = pd.DataFrame(data)

print(df)

Aplica el algoritmo al juego de datos generado

Respuesta

In [ ]:
    
# implementacion del algoritmo para generar una poblacion de 1000 soluciones y escoger la mejor 'menor costo'
# ejecutar la celda precedente para actualizar en contenido de df

# cantidad de solutiones a generar
num_solutions = 1000

# lista para alamcenar las soluciones
solutions = []

# generar las soluciones
for i in range(num_solutions):
    
    full_actor_participation, Recordig_Schedule_Full, actors_recording_total, cost = Generate_solution(df, communality_df)
    
    
    solution = {
        'full_actor_participation': full_actor_participation,
        'Recordig_Schedule_Full': Recordig_Schedule_Full,
        'actors_recording_total': actors_recording_total,
        'cost': cost
    }
    solutions.append(solution)

# escoger la solucion con el menor costo
best_solution = min(solutions, key=lambda x: x['cost'])

#mostrar en pantalla
print("Best solution:")
print(best_solution)


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

In [ ]:
Solo los materiales de clase (modulo y clases)

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Pienso que al definir o representar el problema en una forma diferente se pueden vislumbrar estrategias de solucion alternativas.
Si se continua el camino de la genreracion de programaciones de grabacion entereamente aleatorias, se podrin emplear metaheuristicas del tipo colonia de hormigas, recocido simulado, u algoritmos geneticos, para evaluar las mejores soluciones dentro de la poblacion generada. <br>

Podria explorarse la misma estrategia que desarrolle' en este ejercicio mas no ordenando por tomas sino por actores. Quiza ese camino ofrezca mejores soluciones. 